In [3]:
def find_qa_text(question_id):
    try:
        return qa[qa['question_id'] == question_id].iloc[0].raw_data
    except:
        pass

In [4]:
import pandas as pd
train = pd.read_csv("si630w22-hw3-train.csv")
dev = pd.read_csv("si630w22-hw3-dev.csv")
qa = pd.read_csv("si630w22-hw3-data.csv")
qa["raw_data"] = qa["question_text"] + qa["reply_text"]
# qa["raw_data"] = qa["question_text"]
train.drop("annotator_id", axis=1)
train = train[["id","group", "rating"]].groupby(['id','group']).mean()
train = train.reset_index()
train["qa_text"] = train["id"].apply(find_qa_text)
train["text"] = train["group"] * 5 + train["qa_text"] 
dev.drop("annotator_id", axis=1)
dev = dev[["id","group", "rating"]].groupby(['id','group']).mean()
dev = dev.reset_index()
dev["qa_text"] = dev["id"].apply(find_qa_text)
dev["text"] = dev["group"] * 5 + dev["qa_text"]

In [5]:
train

,id,group,rating,qa_text,text
0,t3_n2714y,group_01,5.000000,"Is there someone you turned down in the past, ...",group_01group_01group_01group_01group_01Is the...
1,t3_n2714y,group_03,4.500000,"Is there someone you turned down in the past, ...",group_03group_03group_03group_03group_03Is the...
2,t3_n27873,group_01,3.500000,"What is, in your opinion, the saddest villain ...",group_01group_01group_01group_01group_01What i...
3,t3_n27873,group_05,4.000000,"What is, in your opinion, the saddest villain ...",group_05group_05group_05group_05group_05What i...
4,t3_n27b1e,group_01,3.500000,ELI5: How do we still not know how eels reprod...,group_01group_01group_01group_01group_01ELI5: ...
...,...,...,...,...,...
7257,t3_npg9cv,group_24,3.000000,"People who stayed in toxic relationships, why ...",group_24group_24group_24group_24group_24People...
7258,t3_npg9lh,group_22,2.333333,What is something that rich people do but midd...,group_22group_22group_22group_22group_22What i...
7259,t3_npg9lh,group_25,2.500000,What is something that rich people do but midd...,group_25group_25group_25group_25group_25What i...
7260,t3_npgle3,group_23,2.500000,"Your phone battery is your age, what do you do...",group_23group_23group_23group_23group_23Your p...


In [6]:
dev

,id,group,rating,qa_text,text
0,t3_n26t3x,group_01,5.000000,"Gamer girls, what are your experiences when ga...",group_01group_01group_01group_01group_01Gamer ...
1,t3_n26t3x,group_02,5.000000,"Gamer girls, what are your experiences when ga...",group_02group_02group_02group_02group_02Gamer ...
2,t3_n27j2t,group_01,4.000000,How can I deal with loneliness in a healthier ...,group_01group_01group_01group_01group_01How ca...
3,t3_n27j2t,group_07,4.666667,How can I deal with loneliness in a healthier ...,group_07group_07group_07group_07group_07How ca...
4,t3_n28ch5,group_02,3.333333,Offset panda: is it a scam?I went to the only ...,group_02group_02group_02group_02group_02Offset...
...,...,...,...,...,...
1549,t3_npex8i,group_24,4.333333,Is it wrong or/and bad to write about the same...,group_24group_24group_24group_24group_24Is it ...
1550,t3_npf5lq,group_20,2.666667,"Retail worker, what are your horror stories?So...",group_20group_20group_20group_20group_20Retail...
1551,t3_npf5lq,group_24,3.000000,"Retail worker, what are your horror stories?So...",group_24group_24group_24group_24group_24Retail...
1552,t3_npg22s,group_21,1.000000,How long would your dog survive on its own in ...,group_21group_21group_21group_21group_21How lo...


In [52]:
def find_another_annotate(id, group_name):
    try:
        return dev[(dev["id"] == id) & (dev["group"] != group_name)].rating.iloc[0]
    except:
        return 3

In [53]:
dev[dev["group"] == "group_25"].id

11      t3_n2chub
44      t3_n2ziu4
62      t3_n3ip8c
72      t3_n3n499
102     t3_n4344j
          ...    
1499    t3_nolym3
1505    t3_nor9b3
1511    t3_nouq7x
1517    t3_nox9ml
1547    t3_npe9n2
Name: id, Length: 65, dtype: object

In [54]:
find_another_annotate("t3_n2ziu4", "group_25")

5.0

In [ ]:
class HelpfulnessDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = torch.nn.MSELoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", num_labels=1).to(device)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.03,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=80,
)

In [63]:
import torch
import scipy.stats
from transformers import AutoTokenizer
iter_list = list(range(25))
iter_list.remove(5)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
max_length = 512
corrs_all = []
for i in iter_list:
    if (i+1) >= 10:
        group_name = "group_" + str(i + 1)
    else:
        group_name = "group_0" + str(i + 1)
    print(group_name)
    training = train[train["group"] != group_name]
    dev1 = dev[dev["group"] != group_name]
    dev2 = dev[dev["group"] == group_name]
    dev3 = dev2.copy()
    dev3['rating'] = dev3.apply(lambda x: find_another_annotate(x.id, x.group), axis=1)
    # dev3["rating"] = dev3["group"].apply(find_another_annotate)
    train_texts, train_labels = training["text"].tolist(), training["rating"].tolist()
    test1_texts, test1_labels = dev1["text"].tolist(), dev1["rating"].tolist()
    test2_texts, test2_labels = dev2["text"].tolist(), dev2["rating"].tolist()
    test3_texts, test3_labels = dev3["text"].tolist(), dev3["rating"].tolist()
    train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=max_length)
    test1_encodings = tokenizer(test1_texts, truncation=True, padding="max_length", max_length=max_length)
    test2_encodings = tokenizer(test2_texts, truncation=True, padding="max_length", max_length=max_length)
    test3_encodings = tokenizer(test3_texts, truncation=True, padding="max_length", max_length=max_length)
    train_dataset = HelpfulnessDataset(train_encodings, train_labels)
    test1_dataset = HelpfulnessDataset(test1_encodings, test1_labels)
    test2_dataset = HelpfulnessDataset(test2_encodings, test2_labels)
    test3_dataset = HelpfulnessDataset(test3_encodings, test3_labels)
    test_datasets = [test1_dataset, test2_dataset, test3_dataset]
    test
    trainer = CustomTrainer(
        model=model,                         # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        # eval_dataset=test_dataset,             # evaluation dataset
        compute_metrics=compute_metrics
    )
    trainer.train()
    corrs = []
    for i in range(3):
        trainer.eval_dataset=test_datasets[i]
        result = trainer.predict(test_datasets[i]).predictions
        corr = (scipy.stats.pearsonr(result, test_datasets[i]["label"])
        corrs.append(corr)
    corrs_all.appedn(corrs)

group_01
group_02
group_03
group_04
group_05
group_07
group_08
group_09
group_10
group_11
group_12
group_13
group_14
group_15
group_16
group_17
group_18
group_19
group_20
group_21
group_22
group_23
group_24
group_25
